In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'C:\Users\shars\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, plot_confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [3]:
dataframe_path_train = "train.csv"
dataframe_path_test = "test.csv"

dataframe_train = pd.read_csv(dataframe_path_train)
dataframe_test = pd.read_csv(dataframe_path_test)

In [4]:
for a in dataframe_test:
    print(dataframe_test[a].value_counts().max(), '--', dataframe_test[a].value_counts().idxmax())

58 -- 0
10 -- 65.0
47 -- 0
13 -- 582
48 -- 0
13 -- 25
56 -- 0
5 -- 263358.03
17 -- 1.0
14 -- 136
49 -- 1
55 -- 0
4 -- 245


In [5]:
list_nan = ["", " "]

def normalize_nan(dataframe, list_nan):
    for elem in list_nan:
        dataframe.replace(elem, np.nan, inplace=True)
    return dataframe

dataframe_train = normalize_nan(dataframe_train, list_nan)
dataframe_test = normalize_nan(dataframe_test, list_nan)

In [6]:
le = LabelEncoder()

target_feature = "DEATH_EVENT"
dataframe_train_target = le.fit_transform(dataframe_train.pop(target_feature))
dataframe_test_target = le.transform(dataframe_test.pop(target_feature))

In [7]:
dataframe_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,58.0,0,132,1,38,1,253000.00,1.00,139,1,0,230
1,43.0,1,358,0,50,0,237000.00,1.30,135,0,0,97
2,42.0,0,5209,0,30,0,226000.00,1.00,140,1,1,87
3,55.0,0,66,0,40,0,203000.00,1.00,138,1,0,233
4,94.0,0,582,1,38,1,263358.03,1.83,134,1,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...
155,58.0,1,133,0,60,1,219000.00,1.00,141,1,0,83
156,50.0,1,54,0,40,0,279000.00,0.80,141,1,0,250
157,44.0,0,582,1,30,1,263358.03,1.60,130,1,1,244
158,50.0,1,168,0,38,1,276000.00,1.10,137,1,0,11


In [8]:
dataframe_test

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,79.0,1,55,0,50,1,172000.0,1.8,133,1,0,78
1,45.0,0,2060,1,60,0,742000.0,0.8,138,0,0,278
2,70.0,0,1202,0,50,1,358000.0,0.9,141,0,0,196
3,50.0,0,369,1,25,0,252000.0,1.6,136,1,0,90
4,51.0,0,78,0,50,0,406000.0,0.7,140,1,0,79
...,...,...,...,...,...,...,...,...,...,...,...,...
74,77.0,1,109,0,50,1,406000.0,1.1,137,1,0,209
75,61.0,1,84,0,40,1,229000.0,0.9,141,0,0,110
76,60.0,0,253,0,35,0,279000.0,1.7,140,1,0,250
77,45.0,0,582,1,55,0,543000.0,1.0,132,0,0,250


In [9]:

def val_to_class(df):
    for elem in df:
        if type(df[elem][1]) is str:
            df[elem] = pd.Categorical(df[elem])
            df[elem] = getattr(df, elem).cat.codes
    return df

dataframe_train = val_to_class(dataframe_train)
dataframe_test = val_to_class(dataframe_test)

In [10]:
dataframe_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,58.0,0,132,1,38,1,253000.00,1.00,139,1,0,230
1,43.0,1,358,0,50,0,237000.00,1.30,135,0,0,97
2,42.0,0,5209,0,30,0,226000.00,1.00,140,1,1,87
3,55.0,0,66,0,40,0,203000.00,1.00,138,1,0,233
4,94.0,0,582,1,38,1,263358.03,1.83,134,1,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...
155,58.0,1,133,0,60,1,219000.00,1.00,141,1,0,83
156,50.0,1,54,0,40,0,279000.00,0.80,141,1,0,250
157,44.0,0,582,1,30,1,263358.03,1.60,130,1,1,244
158,50.0,1,168,0,38,1,276000.00,1.10,137,1,0,11


In [11]:
dataframe_test

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,79.0,1,55,0,50,1,172000.0,1.8,133,1,0,78
1,45.0,0,2060,1,60,0,742000.0,0.8,138,0,0,278
2,70.0,0,1202,0,50,1,358000.0,0.9,141,0,0,196
3,50.0,0,369,1,25,0,252000.0,1.6,136,1,0,90
4,51.0,0,78,0,50,0,406000.0,0.7,140,1,0,79
...,...,...,...,...,...,...,...,...,...,...,...,...
74,77.0,1,109,0,50,1,406000.0,1.1,137,1,0,209
75,61.0,1,84,0,40,1,229000.0,0.9,141,0,0,110
76,60.0,0,253,0,35,0,279000.0,1.7,140,1,0,250
77,45.0,0,582,1,55,0,543000.0,1.0,132,0,0,250


In [12]:
list_classifier = [
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    KNeighborsClassifier(3),
    MLPClassifier(alpha=1, max_iter=1000),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]

for i in range(len(list_classifier)):
    predictions = list_classifier[i]\
            .fit(dataframe_train, dataframe_train_target)\
            .predict(dataframe_test)
    accuracy = metrics.accuracy_score(dataframe_test_target, predictions)
    precision = metrics.precision_score(dataframe_test_target, predictions)
    recall = metrics.recall_score(dataframe_test_target, predictions)
    print(f"precision: {precision:.3} -- recall: {recall:.3} -- accuracy: {accuracy:.3} ")
    


precision: 0.8 -- recall: 0.571 -- accuracy: 0.848 
precision: 0.55 -- recall: 0.524 -- accuracy: 0.759 
precision: 0.824 -- recall: 0.667 -- accuracy: 0.873 
precision: 0.2 -- recall: 0.0952 -- accuracy: 0.658 
precision: 0.35 -- recall: 0.333 -- accuracy: 0.658 
precision: 0.0 -- recall: 0.0 -- accuracy: 0.734 
precision: 0.818 -- recall: 0.429 -- accuracy: 0.823 


C:\Users\shars\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
